In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
import json
import os
from pandas.tseries.offsets import MonthEnd
from plotting import * 
import modify_dataset  
from itertools import combinations
from scipy.stats import entropy
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
import itertools

In [3]:
_HOME_ = path.expanduser("~") + "/PHD"

In [4]:
phyto_abundances = pd.read_csv(_HOME_ + "/MSFD/Data/Modulo1/phyto_abund_modified.csv", index_col=0)
phyto_abundances.loc[:, "Date"] = pd.to_datetime(phyto_abundances["Date"])
with open(_HOME_ + "/ISPRA_20152017_Analysis/params.json") as file: 
    params = json.load(file)
best_path = params["best_path"]
best_path_sard = params["best_path_sard"]
ordered_regions = params["ordered_regions"]
seasons = params["seasons"]
sorted_season = params["sorted_season"]
sea_index = params["sea_index"]
ordered_id = params["ordered_id"]
sea_index_array = np.array(list(sea_index.values()))
del params
phyto_abund_simplified = modify_dataset.make_simplified_dataset(phyto_abundances, 0.7)
phyto_abund_simplified = modify_dataset.add_season_column(phyto_abund_simplified, seasons)
phyto_abund_simplified = modify_dataset.add_coast_dist_column(phyto_abund_simplified)
phyto_abund_simplified["Date"] = pd.to_datetime(phyto_abund_simplified["Date"]) + MonthEnd(0)
phyto_abund_simplified["Region"] = pd.Categorical(phyto_abund_simplified["Region"], categories = ordered_regions, ordered = True)
taxonomic_tree = pd.read_excel(_HOME_ + "/ISPRA_20152017_Analysis/Phyto_taxonomic_tree.xlsx")
taxonomic_tree.drop_duplicates(inplace=True)
phyto_abund_simplified = phyto_abund_simplified.merge(taxonomic_tree.loc[:,["ScientificName", "Genus", "Class"]], how = "left", left_on="Taxon", right_on="ScientificName")
phyto_abund_simplified["Class"] = phyto_abund_simplified["Class"].astype(str)
phyto_abund_simplified["Genus"] = phyto_abund_simplified["Genus"].astype(str)
phyto_abund_simplified = modify_dataset.add_det_level_column(phyto_abund_simplified)
phyto_abund_simplified = phyto_abund_simplified.loc[:,
                                                    ["Region", "id", "Longitude", "Latitude", "Closest_coast", "Coast_dist", "Date", "Season_year", "Season", "Sample_depth", "Class", "Genus", "Det_level", "Taxon", "Num_cell_l", "file_name"]
                                                    ]
phyto_abund_simplified.loc[phyto_abund_simplified["Class"] == "nan", "Class"] = phyto_abund_simplified.loc[phyto_abund_simplified["Class"] == "nan", "Taxon"]
#| phyto_abund_simplified["Genus"].isna(), "Genus"
phyto_abund_simplified.loc[(phyto_abund_simplified["Genus"] == "nan"), "Genus"] = phyto_abund_simplified.loc[(phyto_abund_simplified["Genus"] == "nan"), "Taxon"]
higher_than_class = ["Haptophyta", "Noctilucea", "Non flagellates", "Flagellates"]
phyto_abund_simplified = phyto_abund_simplified.query("Taxon not in @higher_than_class")
is_pseudo_nitzschia = phyto_abund_simplified["Taxon"].isin(["Pseudo-nitzschia spp. del nitzschia delicatissima complex", "Pseudo-nitzschia spp. del nitzschia seriata complex"])
phyto_abund_simplified.loc[is_pseudo_nitzschia, "Taxon"] = "Pseudo-nitzschia spp."
phyto_abund_simplified = phyto_abund_simplified.groupby(["Region", "Season", "Date", "id", "Det_level", "Class", "Genus", "Taxon"], observed=True).sum(numeric_only=True).reset_index()

In [6]:
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from scipy.cluster.hierarchy import fcluster

def build_linkage_matrix(model) -> np.ndarray : 
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    return np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

In [45]:
pres_abs = phyto_abund_simplified.query("Taxon != 'Other phytoplankton' and Region != 'Basilicata'").pivot_table(index = ["id", "Date"], columns="Taxon", values="Num_cell_l", aggfunc="sum", fill_value=0)

In [46]:
pres_abs[pres_abs > 0] = 1

In [47]:
pres_abs = pres_abs.loc[:, pres_abs.sum(axis = 0) > 22]

In [40]:
from scipy.spatial.distance import pdist, squareform

In [51]:
pres_abs_dist = squareform(pdist(pres_abs.to_numpy(), metric="braycurtis"))

In [56]:
pres_abs_dist.shape

(2183, 2183)

In [59]:
cluster_index = {}
single_linkage_clustering = AgglomerativeClustering(n_clusters = None, metric = "precomputed", linkage = "average", compute_distances = True, compute_full_tree=True, distance_threshold=0.0)
linkage_clusters = single_linkage_clustering.fit(pres_abs_dist)
linkage_matrix = build_linkage_matrix(linkage_clusters)
for n in range(2,13):
    cluster_index[f"ward_{n}"] = fcluster(linkage_matrix, t = n, criterion = "maxclust")